In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:\\Users\\SisLab\\!SOYEON\\train.csv")
df.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [3]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
# center_type
data_cat = ohe.fit_transform(df[['center_id']])
df_center = pd.concat([df.drop(columns=['center_id']),pd.DataFrame(data_cat, columns=['center_id' + str(col) for col in ohe.categories_[0]])], axis=1)

# category
data_cat = ohe.fit_transform(df[['meal_id']])
train_df = pd.concat([df_center.drop(columns=['meal_id']),pd.DataFrame(data_cat, columns=['meal_id' + str(col) for col in ohe.categories_[0]])], axis=1)

train_df.head()

,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,center_id10,center_id11,center_id13,...,meal_id2581,meal_id2631,meal_id2640,meal_id2664,meal_id2704,meal_id2707,meal_id2760,meal_id2826,meal_id2867,meal_id2956
0,1379560,1,136.83,152.29,0,0,177,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1466964,1,136.83,135.83,0,0,270,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1346989,1,134.86,135.86,0,0,189,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1338232,1,339.50,437.53,0,0,54,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1448490,1,243.50,242.50,0,0,40,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from sklearn.model_selection import train_test_split

X = train_df.drop('num_orders', axis=1)
y = train_df['num_orders']

In [5]:
# train, test를 8:2로 나누기
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=1)

In [6]:
# shape 확인
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(365238, 134)
(91310, 134)
(365238,)
(91310,)


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_x)
X_train_transformed = pd.DataFrame(scaler.transform(train_x))
print("\nMinMaxScaler: \n", X_train_transformed.describe())


MinMaxScaler: 
                  0              1              2              3    \
count  365238.000000  365238.000000  365238.000000  365238.000000   
mean        0.500463       0.512199       0.381322       0.368321   
std         0.288605       0.288422       0.177079       0.198021   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.250648       0.263889       0.261763       0.232020   
50%         0.500666       0.520833       0.339326       0.314556   
75%         0.750459       0.763889       0.512290       0.497534   
max         1.000000       1.000000       1.000000       1.000000   

                 4              5              6             7    \
count  365238.000000  365238.000000  365238.000000  365238.00000   
mean        0.081136       0.108822       0.015412       0.01504   
std         0.273044       0.311416       0.123184       0.12171   
min         0.000000       0.000000       0.000000       0.00000   
25%         0.000000 

# Random Forest

In [8]:
# 모델 선언
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rfr = RandomForestRegressor()

In [9]:
rfr.fit(train_x,train_y)

RandomForestRegressor()

In [10]:
test_pred = rfr.predict(test_x)
test_pred

array([ 24.48, 391.76,  80.86, ..., 233.53, 616.98,  30.19])

In [11]:
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, test_pred, squared = True)**0.5

166.79766195356905

In [12]:
from sklearn.metrics import r2_score
print(r2_score(test_y, test_pred))

0.8220200347072915


# Grid Search

In [13]:
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [100],
           'min_samples_leaf' : [1],
           'min_samples_split' : [2],
          'max_features' : [1]
            }

In [14]:
gs_s = GridSearchCV(rfr, param_grid = params,  scoring = 'neg_mean_absolute_error')
gs_s.fit(train_x, train_y)

print("Best Parameters : ", gs_s.best_params_)
print("Best Score : ", gs_s.best_score_)

Best Parameters :  {'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best Score :  -75.12358484851302


In [15]:
test_gfs = gs_s.predict(test_x)
test_gfs

array([ 39.62, 255.45,  78.46, ..., 261.95, 565.63,  36.01])

In [16]:
mean_squared_error(test_y, test_gfs, squared = True)**0.5

168.8963519518528

In [17]:
print(r2_score(test_y, test_gfs))

0.8175130810747244


# Random Search

In [18]:
from sklearn.model_selection import RandomizedSearchCV

In [19]:
rfg = RandomForestRegressor()

# Random Search 설정.
rs_model = RandomizedSearchCV(rfg, params)

In [20]:
rs_model.fit(train_x, train_y)

C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=RandomForestRegressor(),
                   param_distributions={'max_features': [1],
                                        'min_samples_leaf': [1],
                                        'min_samples_split': [2],
                                        'n_estimators': [100]})

In [21]:
test_rs = rs_model.predict(test_x)
test_rs

array([ 28.38, 261.57,  83.49, ..., 255.22, 546.1 ,  38.36])

In [22]:
mean_squared_error(test_y, test_rs, squared = True)**0.5

169.92475371873428

In [23]:
print(r2_score(test_y, test_rs))

0.8152840071594207


In [24]:
print("RandomForest 점수비교")
print("---낮을수록 정확도가 높음---")
print("RandomForest : ", mean_squared_error(test_y, test_pred, squared = True)**0.5)
print("Grid Search : ", mean_squared_error(test_y, test_gfs, squared = True)**0.5)
print("Random Search : ", mean_squared_error(test_y, test_rs, squared = True)**0.5)

RandomForest 점수비교
---낮을수록 정확도가 높음---
RandomForest :  166.79766195356905
Grid Search :  168.8963519518528
Random Search :  169.92475371873428
